<a href="https://colab.research.google.com/github/datapreparation-javeriana/etl-tutorial/blob/master/mongodb-to-bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MongoDB to BigQuery (ETL) tutorial

Data Management course  
Univerisdad Javeriana  
March, 2023

Dataset: AirBnB reviews

In [ ]:
!pip install --upgrade google-cloud-bigquery

In [ ]:
!pip install pandas-gbq

In [ ]:
!pip install db-dtypes

In [ ]:
import random
import string

import pymongo

import numpy as np
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    return "".join(random.choice(letters) for i in range(length))

## 1. E: Extracting data

In [ ]:
DB_NAME = "sample_airbnb"
COLLECTION = "listingsAndReviews"

In [ ]:
client = pymongo.MongoClient("mongodb+srv://<user>:<password>@cluster0.<cluster_id>.mongodb.net/?retryWrites=true&w=majority", server_api=pymongo.server_api.ServerApi("1"))

In [ ]:
db = client[DB_NAME]

In [ ]:
col = db[COLLECTION]

In [ ]:
df = pd.DataFrame(list(col.find({}, {"_id": 0, "address.country": 1, "address.market": 1, "price": 1})))

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.head()

## 2. T: Transforming data

In [ ]:
df = pd.concat([
    df.drop("address", axis=1),
    pd.json_normalize(df["address"])
], axis=1)

In [ ]:
df["price"] = pd.to_numeric(df["price"].astype(str))

In [ ]:
df.loc[df["market"] == "", "market"] = "Other"

In [ ]:
df.head()

In [ ]:
df_grouped = df.groupby(["country", "market"]).agg({"price": np.mean}).reset_index().rename(columns={"price": "price_mean"})

In [ ]:
df_grouped.shape

In [ ]:
df_grouped

## 3. L: Loading data

In [ ]:
credentials = service_account.Credentials.from_service_account_file("./javeriana-dataprep.json", scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [ ]:
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [ ]:
# Creating the job config
job_config = bigquery.LoadJobConfig(
    schema=[
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField("country", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("market", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("price_mean", bigquery.enums.SqlTypeNames.FLOAT64)
    ],
    # Drod and re-create table, if exist
    write_disposition="WRITE_TRUNCATE"
)

In [ ]:
BQ_TABLE_NAME = f"AIRBNB.MARKET_STATS_{get_random_string(4)}"
print(BQ_TABLE_NAME)

In [ ]:
# Sending the job to BigQuery
job = client.load_table_from_dataframe(df_grouped, BQ_TABLE_NAME, job_config=job_config)
job.result()

In [ ]:
# Verifying if table was successfully created or updated
table = client.get_table(BQ_TABLE_NAME)
print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), BQ_TABLE_NAME))

In [ ]:
query = f"""SELECT * FROM `javeriana-378220.{BQ_TABLE_NAME}`"""
pd.read_gbq(query, credentials=credentials)